# AAFormer: Adaptive Agent Transformer for Few-shot Segmentation

**Paper authors:** Yuan Wang, Rui Sun, Zhe Zhang, Tianzhu Zhang

**Reproduced by:** Yusuf Soydan, Bartu Akyürek

This paper is about few-shot segmentation (FSS), aiming to segment objects in a given image and a set of support images with masks.

## Prepare Workspace


In [1]:
!pip install POT
!pip install tqdm
!pip install torchinfo
!pip install matplotlib

In [2]:
# General Dependencies

import os
import math
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torchinfo import summary
from tqdm import tqdm

In [3]:
"""
Run from Colab or local
"""

try:
    from google.colab import drive
    drive.mount('/content/gdrive')
    ROOT_PATH = "/content/gdrive/MyDrive/AAFormer"
    DATA_PATH = os.path.join(ROOT_PATH, "Datasets")

    %cd ./gdrive/MyDrive/AAFormer
    !matplotlib inline

except:
    ROOT_PATH = os.curdir
    DATA_PATH = "../Datasets"
    # Note: If you are running this notebook on your local, please put the Datasets folder outside of current directory

In [4]:
# Source dependencies
from data.dataset import FSSDataset
from model.aaformer import AAFormer

from common.vis import Visualizer
from common.plot import image_plot
from common.dice_loss import DiceLoss
from common.evaluation import Evaluator
from common.dice_loss import BinaryDiceLoss


In [5]:
# HYPERPARAMETERS
# -----------------------------------------------------------------
# Note: the parameters with * comment are NOT provided by the paper
# TODO: make hyperparameters UPPER_CASE
image_resolution = 128
reduce_dim = 256
c = 328             #   Hidden dimensions
hw = 256            # * Dimensionality of Feature maps (depends on ResNet choice)
N = 16              # * Number of layers in Encoder/Decoder
heads = 8           # * Number of attention heads
num_tokens = 14     #   Number of Agent Tokens (see p.13, "Hyperparameter Evaluations")
sinkhorn_reg = 0.05 #   Regularization term of Optimal Transport, (see supplementary)
max_iter_ot = 10    #   Maximum iterations of Optimal Transport algorithm, (see supplementary)
shot = 3

batch_size = 4    # According to paper: 4
adam_lr = 1e-4    # According to paper: 0.0001
adam_decay = 1e-2 # According to paper: 0.01
num_epoch = 20    # According to paper: 400

# Additional parameters to control the flow
# ----------------------------------------------------------------
bypass_ot = False                         # To turn off OT module if it causes any problem (e.g. RAM failure)
use_dice_loss = False                     # False means use CrossEntropy loss
cuda = torch.cuda.is_available()          #
device = 'cuda' if cuda else 'cpu'        #
checkpoint = 100                          # Save the model every {checkpoint} iteration

## Prepare PASCAL-$5^i$ Dataset

In [6]:
# STEP 1: Download PASCAL VOC2012 devkit (train/val data): (uncomment lines below)
# ------------------------------------------------------------------------------
#!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar
#!tar -xvf 'VOCtrainval_11-May-2012.tar' -C ./Datasets/

# (or instead of wget, use directly the link to download)
# STEP 2: Place "VOC2012" folder from downloaded "VOCdevkit" folder under a "Datasets" folder.
# ------------------------------------------------------------------------------
#!mv Datasets/VOCdevkit/VOC2012 Datasets/VOC2012

# STEP 3: Download extended annotations from here
#!wget https://drive.google.com/file/d/10zxG2VExoEZUeyQl_uXga2OWHjGeZaf2/view

# STEP 4: Extract Put the downloaded extension under "Datasets/VOC2012/"


In [7]:
# Dataset initialization
# TODO: does paper mention about image size? TODO: yes, 473
# TODO: they also do data aug...
# TODO: add disclaimer to dataset files https://github.com/juhongm999/hsnet/blob/main/train.py
FSSDataset.initialize(img_size=image_resolution, datapath=DATA_PATH, use_original_imgsize=False)
dataloader_trn = FSSDataset.build_dataloader(benchmark='pascal', bsz=batch_size, shot=shot, nworker=1, fold=0, split='trn')
dataloader_val = FSSDataset.build_dataloader(benchmark='pascal', bsz=batch_size, shot=shot, nworker=1, fold=0, split='val')

Total (trn) images are : 11394
Total (val) images are : 346


## Initialize Models

In [8]:


model = AAFormer(cuda = cuda,
                 c = c,
                 hw = hw,
                 N = N,
                 heads = heads,
                 num_tokens = num_tokens,
                 im_res=image_resolution,
                 reduce_dim=reduce_dim,
                 bypass_ot=bypass_ot,
                 sinkhorn_reg=sinkhorn_reg,
                 max_iter_ot=max_iter_ot)

## Training Loop

In [9]:
# See Supplementary Material: "Dice loss is adopted to train our model" and AdamW optimizer is used for transformer blocks
# Although BCE loss is not mentioned in the paper, many papers working on segmentation uses BCE to compare the binary masks
# Dice loss function we used returns a very low value (about 0.8), which may result in tiny gradients.
dice_loss = DiceLoss() #BinaryDiceLoss()
bce_loss = nn.CrossEntropyLoss()

optimizer = torch.optim.AdamW(model.parameters(), lr=adam_lr, weight_decay=adam_decay)
# TODO: how to seperate optimizers for models and rest?

#####
use_dice_loss = True
####

step = 0
train_losses = []
val_losses = []
for epoch in range(num_epoch):

    model.train()
    train_loss = 0.0
    for idx, batch in tqdm(enumerate(dataloader_trn)):

        # STEP 0: Get query image and support images with corresponding masks
        query_img = batch['query_img'].to(device)
        query_mask = batch['query_mask'].to(device)
        supp_imgs = batch['support_imgs'].to(device)
        supp_masks = batch['support_masks'].to(device)

        # STEP 1: Get predicted mask
        #try:
        preds = model(query_img, supp_imgs, supp_masks, normalize=use_dice_loss) 
        #except:
        #    print(">> Unexpected error occured, skipping this batch...")
        #    continue

        # STEP 2: Compute Dice loss (Note: Dice loss is from rage [0,1])
        # Dice loss and Cross Entropy loss expects [N, C, *] shape,
        # where N is the batch size and C is the number of classes
        # Assumption: we assume C = 1, the only class is mask, i.e. a pixels is either from the
        # mask class or not. Pixels belonging to mask class makes up the segmentation result.
        if use_dice_loss:
            loss = dice_loss(preds, query_mask.unsqueeze(1)) # Make query_mask shape (batchsize,*) --> (batchsize, 1, *)
        else:
            loss = bce_loss(preds, query_mask.unsqueeze(1))  # Make query_mask shape (batchsize,*) --> (batchsize, 1, *)

        train_loss += loss.item()

        # STEP 3: Update optimizer
        step += 1
        loss.backward()
        optimizer.step()

        #### Ploting preds with inputs ####
        image_plot(query_img, supp_imgs, supp_masks, preds)
        print(f"iteration {idx+1} train_loss = {loss.item()}")
        if(idx+1 % 10 == 0):
            print(f"Step:{step} Loss:{loss.item()}")
            #image_plot(query_img, supp_imgs, supp_masks, preds)


        # TODO: save model file at every X iteration
        if(step % checkpoint == 0):
            pass

    # Append the average loss
    train_losses.append(train_loss/len(dataloader_trn))
    print("Epoch ", epoch, " Loss: ", train_loss/len(dataloader_trn))

    # TODO: validation loss? (with torch.no_grad():?)


0it [00:04, ?it/s]


AttributeError: 'FeatureExtractor' object has no attribute 'dtype'

In [ ]:
# TODO plot train/val loss

In [ ]:
image_plot(query_img, supp_imgs, supp_masks, preds)

In [ ]:
preds.shape

In [ ]:
fig = plt.figure()
i = 0
for q in range(query_img.shape[0]):
    plt.subplot(query_img.shape[1],query_img.shape[0],i+1)
    plt.imshow(transform_img(query_img[q,:,:,:]))
    plt.suptitle("query_img", y=0.7)
    plt.axis('off')
    i += 1
plt.show()

i = 0
for s in range(supp_imgs.shape[1]):
    for supp_img in supp_imgs[:,s,:,:]:
        plt.subplot(supp_imgs.shape[1],supp_imgs.shape[0],i+1)
        plt.imshow(transform_img(supp_img))
        plt.suptitle("supp_img", y=0.7)
        plt.axis('off')
        i += 1
plt.show()

i = 0
for s in range(supp_masks.shape[1]):
    for supp_mask in supp_masks[:,s,:,:]:
        plt.subplot(supp_masks.shape[1],supp_masks.shape[0],i+1)
        plt.imshow(transform_mask(supp_mask))
        plt.suptitle("supp_masks", y=0.7)
        plt.axis('off')
        i += 1
plt.show()

## Visualization Test

In [ ]:
""" Code below visualizes prediction (to be used in testing) """

Visualizer.initialize(True)
Evaluator.initialize()

vis_step = 1

for idx, batch in enumerate(dataloader_trn):

        # STEP 0: Get query image and support images with corresponding masks
        query_img = batch['query_img'].to(device)
        supp_imgs = batch['support_imgs'].to(device)
        supp_masks = batch['support_masks'].to(device)

        # STEP 1: Get predicted mask
        model.eval()
        pred_mask = model(query_img, supp_imgs, supp_masks, normalize=use_dice_loss)
        pred_mask = pred_mask.squeeze(1)

        #print(batch['query_mask'].shape)
        #print(pred_mask.shape)
        assert pred_mask.size() == batch['query_mask'].size()


        # 2. Evaluate prediction
        area_inter, area_union = Evaluator.classify_prediction(pred_mask.clone(), batch)

        #average_meter.update(area_inter, area_union, batch['class_id'], loss=None)
        #average_meter.write_process(idx, len(dataloader), epoch=-1, write_batch_idx=1)

        # Visualize predictions
        if Visualizer.visualize:
            Visualizer.visualize_prediction_batch(batch['support_imgs'], batch['support_masks'],
                                                  batch['query_img'], batch['query_mask'],
                                                  pred_mask, batch['class_id'], idx,
                                                  area_inter[1].float() / area_union[1].float())

        if idx+1 == vis_step:
            break



In [ ]:
# TODO: declare numerical results